In [16]:
from gensim import utils
import gensim.models.doc2vec
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
import gensim
import sys
import numpy as np
from gensim import corpora, models
import csv
import pickle
from sklearn.externals import joblib
import bz2
from random import shuffle
import ast
from sklearn.linear_model import LogisticRegression

from time import time
import gzip

import multiprocessing

cores = multiprocessing.cpu_count()
assert gensim.models.doc2vec.FAST_VERSION > -1, "multi-threading"
print(cores)

8


* min_count: ignore all words with total frequency lower than this. You have to set this to 1, since the sentence labels only appear once. Setting it any higher than 1 will miss out on the sentences.

* window: the maximum distance between the current and predicted word within a sentence. Word2Vec uses a skip-gram model, and this is simply the window size of the skip-gram model.

* size: dimensionality of the feature vectors in output. 100 is a good number. If you're extreme, you can go up to around 400.

* sample: threshold for configuring which higher-frequency words are randomly downsampled

* workers: use this many worker threads to train the model

In [39]:
%%time


doc_filepath = 'data/eos/eos_text_tokenize.gz'
doc2vec_model_path = 'data/eos/doc2vec_model.d2v'

if 0 == 1:
    sentences = gensim.models.doc2vec.TaggedLineDocument(doc_filepath)
    
    token_count = sum([len(sentence) for sentence in sentences])

    model = Doc2Vec(min_count=10, window=10, size=100, sample=1e-4, workers=7)  # use fixed learning rate
    
    model.build_vocab(sentences)
    
    print("done building vocab")
    
    for epoch in range(10):
#         time = time()
        print("starting epoch")
        model.train(sentences, total_examples = token_count, epochs = model.iter)
        model.alpha -= 0.002  # decrease the learning rate
        model.min_alpha = model.alpha  # fix the learning rate, no decay
        print('Epoch took %.2f seconds to run.')
    
    model.save(doc2vec_model_path)
    
else:
    model = Doc2Vec.load(doc2vec_model_path)

done building vocab
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
starting epoch
Epoch took %.2f seconds to run.
CPU times: user 6min 36s, sys: 10 s, total: 6min 46s
Wall time: 2min 37s


NoneType


In [40]:
sims = model.docvecs.most_similar(99)
print(sims)

[(5350, 0.8358443975448608), (759, 0.5828502774238586), (1110, 0.579675555229187), (3006, 0.5721727013587952), (5280, 0.5715876221656799), (7416, 0.57147216796875), (2746, 0.5673331022262573), (1828, 0.566329836845398), (6538, 0.5659772753715515), (2184, 0.5656006336212158)]


In [41]:
print(model.doesnt_match("iran turkey".split()))

iran


In [42]:
print(model.most_similar(positive=['saudi', 'king'], negative=['girl']))

[('riyadh', 0.45443204045295715), ('kaec', 0.3976063132286072), ('arabia', 0.3755999207496643), ('prince', 0.37310129404067993), ('kingpin', 0.3690277338027954), ('ally', 0.3533684015274048), ('thanked', 0.3499802350997925), ('arabian', 0.34905827045440674), ('conceived', 0.34479257464408875), ('else', 0.3429468870162964)]


In [49]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

textVect = model.docvecs.doctag_syn0

## K-means ##
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(textVect)
clusters = km.labels_.tolist()

print(clusters)

plt.scatter(km[:, 0], km[:, 1], c=clusters)
plt.title("Anisotropicly Distributed Blobs")

[2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 2, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 1, 2, 0, 1, 0, 1, 2, 1, 1, 0, 0, 1, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 0, 1, 2, 0, 1, 2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 1, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 2, 1, 0, 1, 2, 0, 0, 1, 0, 1, 0, 2, 0, 0, 2, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 2, 0, 1, 2, 2, 0, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 0, 1, 2, 0, 0, 2, 0, 1, 0, 2, 2, 1, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 2, 1, 2, 0, 0, 2, 0, 0, 1, 0, 0, 2, 1, 0, 0, 2, 2, 2, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 2, 0, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 1, 1, 1, 2, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 

TypeError: 'KMeans' object is not subscriptable